In [1]:
import numpy as np
import pulp

In [2]:
# sample cycles
samp_cycles = \
[[2, 7, 31, 27],
 [2, 31, 8, 11, 27],
 [2, 31, 8, 28, 27],
 [2, 31, 27],
 [7, 31, 8, 26],
 [7, 31, 18, 24, 26],
 [7, 31, 18, 26],
 [7, 31, 27, 26],
 [7, 41],
 [7, 41, 35, 8, 26],
 [7, 41, 35, 26],
 [8, 11, 17, 41, 35],
 [8, 11, 17, 49, 16],
 [8, 26, 36, 49, 16],
 [8, 28, 31],
 [11, 17, 29, 24, 26],
 [11, 17, 41, 35, 26],
 [11, 17, 49],
 [11, 27, 26],
 [11, 27, 26, 36, 49],
 [16, 41, 35],
 [26, 36]]

In [3]:
# abstracted, but we would be given the cycle cost pairs
cycle_cost_pairs = [(tuple(cycle), np.random.randint(10)) for cycle in samp_cycles]

In [4]:
print cycle_cost_pairs

[((2, 7, 31, 27), 1), ((2, 31, 8, 11, 27), 4), ((2, 31, 8, 28, 27), 6), ((2, 31, 27), 2), ((7, 31, 8, 26), 0), ((7, 31, 18, 24, 26), 7), ((7, 31, 18, 26), 1), ((7, 31, 27, 26), 5), ((7, 41), 2), ((7, 41, 35, 8, 26), 1), ((7, 41, 35, 26), 5), ((8, 11, 17, 41, 35), 2), ((8, 11, 17, 49, 16), 5), ((8, 26, 36, 49, 16), 6), ((8, 28, 31), 1), ((11, 17, 29, 24, 26), 3), ((11, 17, 41, 35, 26), 6), ((11, 17, 49), 6), ((11, 27, 26), 3), ((11, 27, 26, 36, 49), 1), ((16, 41, 35), 0), ((26, 36), 8)]


In [5]:
# extract cycles and costs from the given information
cycles, costs = zip(*cycle_cost_pairs)

In [6]:
# create a set of all individuals
patients = set()
[patients.update(cycle) for cycle in cycles]
print patients

set([2, 35, 36, 49, 7, 8, 41, 11, 16, 17, 18, 24, 26, 27, 28, 29, 31])


In [7]:
def cycleCost(cycle):
    """Calculate the cost of a particular cycle"""
    return cycle[1]

In [8]:
#create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts('surgery_group', cycle_cost_pairs,
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

In [9]:
surgery_model = pulp.LpProblem("Kidney Donor/Patient Model", pulp.LpMaximize)

In [10]:
# objective function
surgery_model += sum([cycleCost(cycle) * x[cycle] for cycle in cycle_cost_pairs])

In [11]:
# constraint that patient must appear no more than once
for patient in patients:
    surgery_model += sum([x[cycle] for cycle in cycle_cost_pairs
                                if patient in cycle[0]]) <= 1 #, "Must_operate_{0}".format(patient)

In [12]:
surgery_model.solve()

1

In [13]:
print("The choosen surgery cycles are out of a total of {0} cycles and {1} patients:".format(len(cycle_cost_pairs),
                                                                                                 len(patients)))
for cycle in cycle_cost_pairs:
    if x[cycle].value() == 1.0:
        print(cycle[0])

The choosen surgery cycles are out of a total of 22 cycles and 17 patients:
(2, 31, 8, 28, 27)
(7, 41)
(11, 17, 49)
(26, 36)
